In [1]:
import open3d as o3d
import laspy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [19]:
#used to load .las data into a laspy object.
def load_laspy(las_location):
      input_las = laspy.read(las_location)
      
      return input_las

#used to return a list of unique classifications present in the input LAS file.
def get_list_classifications(input_las):
    classifications = list(set(input_las.classification))
    classifications.sort()
    
    return classifications

#Used to return a Python dictionary of laspy objects of the entire dataset as well as individual classifications.
def get_classifications_laspy(input_las):
    #get a unique list of the classifications currently contained in the dataset.
    classifications = get_list_classifications(input_las=input_las)
    #create dictionary with complete data for each province
    d = {"original_dataset": input_las}
    print("The classifications contained in this .las dataset are: ")
    print(classifications)
    print("Creating a python dictionary containing the entire dataset and individual classifications as separate laspy objects.")
    for classification in classifications:
        print("Saving classification number: " +str(classification))
        classified_points = laspy.create(point_format=input_las.header.point_format, file_version=input_las.header.version)
        classified_points.points = input_las.points[input_las.classification == classification]
        
        d["classification_{}".format(str(classification))] = classified_points
    
    return d

#Optional function which takes the dictionary of laspy objects and converts every classification
#   into individual .LAS files 
def laspy_classifications_to_las(classifications_laspy):
    for classification_las in classifications_laspy:
        if classification_las == "original_dataset":
            pass
            print("skipped exporting the original dataset.")
        else:
            print("processing {} to a LAS file: ".format(classification_las))
            print(classifications_laspy[classification_las])
            classifications_laspy[classification_las].write("classification_{}.las".format(classification_las.split("_")[1]))
            print("done")

# Convert a .LAS file into a pandas object.
def convert_laspy_pandas(input_las):
    # Convert data into pandas DataFrame
    df = pd.DataFrame({"X":input_las.X,"Y":input_las.Y,"Z":input_las.Z,
      "x":np.array(input_las.x),"y":np.array(input_las.y),"z":np.array(input_las.z),
     'intensity': input_las.intensity,
      'classification': input_las.classification,
      'return_number': np.array(input_las.return_number),
      'number_of_returns':np.array(input_las.number_of_returns),
      'synthetic':np.array(input_las.synthetic),
      'key_point':np.array(input_las.key_point),
      'withheld':np.array(input_las.withheld),
      'overlap':np.array(input_las.overlap),
      'scanner_channel':np.array(input_las.scanner_channel),
      'scan_direction_flag':np.array(input_las.scan_direction_flag),
      'user_data':input_las.user_data,
      'scan_angle':input_las.scan_angle,
      'point_source_id':input_las.point_source_id,
      'gps_time':input_las.gps_time    
      })

    return df

def get_selected_classes(classifications,input_las):
    classes = get_list_classifications(input_las=input_las)
    available_classes = classes
    available_classes.append(1000) #to represent the entire dataset
    class_selections = []
    current_selection = -10

    while current_selection == -10:
        print("Available classifications: " + str(available_classes))
        try:
            current_selection = input("Enter a number from the list of available classifications. To get the full dataset enter 1000 : \n")
            if current_selection.isnumeric() == False:
                print("Invalid Character Input.")
                raise ValueError()
            current_selection = int(current_selection)
            if current_selection not in available_classes:
                print("The classification number you selected: {}, is not an available class item.".format(current_selection))
                cancel = input("Would you like to exit instead? 'Y/N' ").upper() + "Y"
                if cancel[0] == "Y":
                    current_selection = -1000
                    print("Exiting and returning empty classification selection list.")
                    class_selections = []    
                else: 
                    raise ValueError()
            if (current_selection != -1000) & (current_selection != 1000):

                if 1000 in available_classes:
                    available_classes.remove(1000)
                class_selections.append(current_selection)
                print("Class {} added to dataframe.".format(str(current_selection)))
                available_classes.remove(current_selection)
                ask = input("Would you like to add another classification to the dataframe? 'Y/N' ").upper() + "Y"

                #If user prompts to add another classification, reset variable and start again.
                if ask[0] == "Y":
                    current_selection = -10
                #If user prompts to stop, exit loop and create pandas dataframe.
                else:
                    print("Classifications have been selected.")
                    class_selections.sort()

            elif current_selection == 1000:
                print("The entire dataset has been selected")
                class_selections = get_list_classifications(input_las=input_las)
                print("All classes have been added to the dataframe.")
                class_selections.sort()

        except ValueError:
            print("Please select an input from the available classification list. \n")
            current_selection = -10

    
    return class_selections

def create_combined_classifications_dataframe(class_selections,df):
    if len(class_selections) > 0:
        #initialize the dataframe with the first set of classifications
        combined_df = df.loc[df["classification"] == class_selections[0]]
        #iterate and append remaining classifications 
        for selection in class_selections[1:]:
            combined_df=pd.concat([combined_df, df.loc[df["classification"] == selection]])
    else:
        combined_df = pd.DataFrame()

    return combined_df

def create_laspy_from_dataframe(input_las,combined_df):
    classified_points = laspy.create(point_format=input_las.header.point_format, file_version=input_las.header.version)

    classified_points.X = combined_df["X"]
    classified_points.Y = combined_df["Y"]
    classified_points.Z = combined_df["Z"]
    classified_points.intensity = combined_df["intensity"]
    classified_points.classification = combined_df["classification"]

    return classified_points

def get_numpy_points_from_laspy_scaled(classified_points):
    point_records = classified_points.points.copy()
    # getting scaling and offset parameters
    las_scaleX = classified_points.header.scale[0]
    las_offsetX = classified_points.header.offset[0]
    las_scaleY = classified_points.header.scale[1]
    las_offsetY = classified_points.header.offset[1]
    las_scaleZ = classified_points.header.scale[2]
    las_offsetZ = classified_points.header.offset[2]
    # calculating coordinates
    p_X = np.array((point_records.X * las_scaleX) + las_offsetX) 
    p_Y = np.array((point_records.Y * las_scaleY) + las_offsetY)
    p_Z = np.array((point_records.Z * las_scaleZ) + las_offsetZ)

    classified_points_numpy = np.array(list(zip(p_X,p_Y,p_Z)))

    return classified_points_numpy


def get_numpy_points_from_laspy_unscaled(classified_points):
    point_records = classified_points.points.copy()
    # calculating coordinates
    p_X = np.array(point_records.X)
    p_Y = np.array(point_records.Y)
    p_Z = np.array(point_records.Z)

    classified_points_numpy = np.array(list(zip(p_X,p_Y,p_Z)))

    return classified_points_numpy

def get_numpy_points_from_pandas_unscaled(combined_df):

    classified_points_numpy = np.array(list(zip(np.array(combined_df["X"]),
                                    np.array(combined_df["Y"]),
                                    np.array(combined_df["Z"]))))

    return classified_points_numpy

def get_numpy_points_from_pandas_scaled(combined_df):

    classified_points_numpy = np.array(list(zip(np.array(combined_df["x"]),
                                    np.array(combined_df["y"]),
                                    np.array(combined_df["z"]))))

    return classified_points_numpy

#input numpy array to visualize using open3d
def visualize_las(classified_points,df= pd.DataFrame()):
        
    dataset = classified_points
    geom = o3d.geometry.PointCloud()
    geom.points = o3d.utility.Vector3dVector(dataset)
    #geom.colors = o3d.utility.Vector3dVector(x)
    #print(df)

    if df.empty is False:
        print("entered df")
        #coord = o3d.geometry.TriangleMesh().create_coordinate_frame(size=df.X[0], origin=[df.X.mean(), df.Y.mean(), df.Z.mean()])
        #coord = o3d.geometry.TriangleMesh().create_coordinate_frame(size=0.5, origin=[df.X.mean(),df.Y.mean(),df.Z.mean()])
        coord = o3d.geometry.TriangleMesh().create_coordinate_frame(size=0.5, origin=[0,0,0])
        o3d.visualization.draw_geometries([coord,geom])
        #o3d.visualization.draw_geometries([geom])

    else:
        o3d.visualization.draw_geometries([geom])
        print("did not enter df")


def create_laspy_from_dataframe(input_las,combined_df):
    classified_points = laspy.create(point_format=input_las.header.point_format, file_version=input_las.header.version)
    
    classified_points.X = combined_df["X"]
    classified_points.Y = combined_df["Y"]
    classified_points.Z = combined_df["Z"]
    classified_points.intensity = combined_df["intensity"]
    classified_points.classification = combined_df["classification"]

    return classified_points
    

def multiple_classifications_to_las(classified_points,class_selections):
    ##write to a .las file
    class_selections.sort()
    sorted_classes_str = "_".join(map(str,class_selections))
    las_file_name = "classifications_" + sorted_classes_str +".las"
    classified_points.write(las_file_name)
    print("Created file: " + sorted_classes_str)




In [26]:
def create_laspy_from_dataframe(input_las,combined_df):
    classified_points = laspy.create(point_format=input_las.header.point_format, file_version=input_las.header.version)
    
    classified_points.X = combined_df["X"]
    classified_points.Y = combined_df["Y"]
    classified_points.Z = combined_df["Z"]
    classified_points.intensity = combined_df["intensity"]
    classified_points.classification = combined_df["classification"]

    return classified_points
    

def multiple_classifications_to_las(classified_points,class_selections):
    ##write to a .las file
    class_selections.sort()
    sorted_classes_str = "_".join(map(str,class_selections))
    las_file_name = "classifications_" + sorted_classes_str +".las"
    classified_points.write(las_file_name)
    print("Created file: " + sorted_classes_str)

In [37]:
input_las.classification

array([9, 9, 9, ..., 0, 0, 0], dtype=uint8)

In [101]:

import pyproj

input_las = load_laspy('Tile65_Original_Clean_Macro.las')

classified_points = laspy.create(point_format=input_las.header.point_format, file_version=input_las.header.version)

classified_points.X = input_las.X
classified_points.Y = input_las.Y
classified_points.Z = input_las.Z
classified_points.intensity = input_las.intensity
classified_points.classification = input_las.classification



classified_points.write("simple_transfer.las")

In [108]:
classified_points.header.offsets

array([0., 0., 0.])

<LasData(1.4, point fmt: <PointFormat(6, 0 bytes of extra dims)>, 1475679 points, 0 vlrs)>

In [118]:
import laspy
import numpy as np
import pyproj

# Define the EPSG code of your desired CRS
#crs_epsg = 2961  # example: UTM Zone 20N
crs = pyproj.CRS.from_epsg(4269)  # example: WGS84



input_las = load_laspy('Tile65_Original_Clean_Macro.las')


# 1. Create a new header
header = laspy.LasHeader(point_format=input_las.header.point_format, version=input_las.header.version)
#header.add_extra_dim(laspy.ExtraBytesParams(name="random", type=np.int32))
#header.offsets = np.min(my_data, axis=0)
header.scales = np.array([0.1, 0.1, 0.1])



# 2. Create a Las
classified_points = laspy.LasData(header)
#classified_points.X = input_las.X
#classified_points.Y = input_las.Y
#classified_points.Z = input_las.Z
classified_points.x = input_las.x
classified_points.y = input_las.y
classified_points.z = input_las.z
#classified_points.xyz = input_las.xyz
classified_points.classification = input_las.classification
classified_points.intensity = input_las.intensity
classified_points.header.add_crs(crs)

classified_points.header.offsets = input_las.header.offsets

#classified_points.header.x_offset = input_las.X.mean()
#classified_points.header.y_offset = input_las.Y.mean()
#classified_points.header.x_offset = input_las.Z.mean()



classified_points.write("new_Test_file2.las")

In [79]:
input_las = load_laspy('new_Test_file2.las')
df = convert_laspy_pandas(input_las=input_las)


In [90]:
input_las.header.

<LasHeader(1.4, <PointFormat(6, 0 bytes of extra dims)>)>

In [80]:
df

,X,Y,Z,x,y,z,intensity,classification,return_number,number_of_returns,synthetic,key_point,withheld,overlap,scanner_channel,scan_direction_flag,user_data,scan_angle,point_source_id,gps_time
0,290988240,-186802461,-278,29098824.0,-18680246.1,-27.8,2474,9,0,0,0,0,0,0,0,0,0,0,0,0.0
1,290987469,-186802282,-237,29098746.9,-18680228.2,-23.7,1869,9,0,0,0,0,0,0,0,0,0,0,0,0.0
2,290987360,-186802266,-267,29098736.0,-18680226.6,-26.7,4226,9,0,0,0,0,0,0,0,0,0,0,0,0.0
3,290987251,-186802246,-286,29098725.1,-18680224.6,-28.6,16268,9,0,0,0,0,0,0,0,0,0,0,0,0.0
4,290987142,-186802229,-314,29098714.2,-18680222.9,-31.4,2284,9,0,0,0,0,0,0,0,0,0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475674,291014354,-186957702,-13723,29101435.4,-18695770.2,-1372.3,1116,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1475675,291017691,-186957217,-15297,29101769.1,-18695721.7,-1529.7,872,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1475676,291013483,-186958110,-13518,29101348.3,-18695811.0,-1351.8,799,0,0,0,0,0,0,0,0,0,0,0,0,0.0
1475677,291018905,-186957521,-15309,29101890.5,-18695752.1,-1530.9,870,0,0,0,0,0,0,0,0,0,0,0,0,0.0


In [ ]:
import laspy
import pyproj

# Define the EPSG code of your desired CRS
crs_epsg = 26910  # example: UTM Zone 10N

# Get the projection information from the EPSG code
proj = pyproj.Transformer.from_crs("epsg:4326", f"epsg:{crs_epsg}")

# Open the LAS file with laspy
las = laspy.read("example.las")

# Set the CRS information in the LAS header
las.header.set_scale(proj.a, proj.b, proj.c)
las.header.set_offset(proj.xoff, proj.yoff, proj.zoff)
las.header.set_srs(pyproj.crs.CRS.from_epsg(crs_epsg).to_wkt())

# Close the LAS file
las.close()

In [21]:
# Read .las file with the load_laspy() function. Note we cannot handle .laz data.
input_las = load_laspy('Tile65_Original_Clean_Macro.las')
#input_las = load_laspy("200-145324069_ground.las")
#Obtain the classifications available for the dataset
classifications = get_list_classifications(input_las=input_las)
#Create a python dictionary containing the original dataset
# as well as classifications in individual laspy objects.
classifications_laspy =  get_classifications_laspy(input_las=input_las)

#If we choose to export each classification into individual .las files.
#laspy_classifications_to_las(classifications_laspy=classifications_laspy)

#Create a pandas dataframe containing the entire pointcloud.
df = convert_laspy_pandas(input_las=input_las)
#Create a pandas dataframe for ground class "2"
if len(df[df["classification"] == 2]) > 0:
    ground = df[df["classification"] == 2]
#Create a pandas dataframe for low veg class "3"
if len(df[df["classification"] == 3]) > 0:
    low_veg = df[df["classification"] == 3]
#Create a pandas dataframe for med veg class "4"
if len(df[df["classification"] == 4]) > 0:
    med_veg = df[df["classification"] == 4]
#Create a pandas dataframe for high veg class "5"
if len(df[df["classification"] == 5]) > 0:
    high_veg = df[df["classification"] == 5]
#Create a pandas dataframe for high veg class "7"
if len(df[df["classification"] == 7]) > 0:
    low_noise = df[df["classification"] == 7]
#Create a pandas dataframe for high veg class "9"
if len(df[df["classification"] == 9]) > 0:
    topo_water_surface = df[df["classification"] == 9]
#Create a pandas dataframe for high noise "18"
if len(df[df["classification"] == 18]) > 0:
    high_noise = df[df["classification"] == 18]
#Create a pandas dataframe for classified bathymetry "40"
if len(df[df["classification"] == 40]) > 0:
    bathymetry = df[df["classification"] == 40]
#Create a pandas dataframe for classified bathymetric water surface "41"
if len(df[df["classification"] == 41]) > 0:
    bathymetric_water = df[df["classification"] == 41]
#Create a pandas dataframe for classified derived water surface "42"
if len(df[df["classification"] == 42]) > 0:
    derived_water_surface = df[df["classification"] == 42]
#Create a pandas dataframe for bathymetric vegetation "80"
if len(df[df["classification"] == 80]) > 0:
    bathy_veg = df[df["classification"] == 80]
#To get a pandas dataframe with specified classification values
#Inform user which class values are present 
#Proceed by prompting user input to select classes to group into a final dataframe.
class_selections = get_selected_classes(classifications = classifications,input_las=input_las)
# Form a pandas dataframe from user-selected classifications
combined_df = create_combined_classifications_dataframe(class_selections=class_selections,df=df)

##Create laspy points from the pandas dataframe of selected user classifications
classified_points = create_laspy_from_dataframe(input_las=input_las,combined_df=combined_df)

###                                 ###
###DATA CONVERSION FOR VISUALIZATION###
###             #                   ###
# Convert classified_points (laspy data) or combined_df (pandas data) to numpy data for visualization
#   with the pptk library. 
#Convert from laspy to numpy scaled
classified_points_numpy_scaled_laspy = get_numpy_points_from_laspy_scaled(classified_points=classified_points)
#Convert from laspy to numpy unscaled
classified_points_numpy_unscaled_laspy = get_numpy_points_from_laspy_unscaled(classified_points=classified_points)
#Convert from pandas to numpy scaled
classified_points_numpy_scaled_pandas = get_numpy_points_from_pandas_scaled(combined_df=combined_df)
#Convert from pandas to numpy unscaled
classified_points_numpy_unscaled_pandas = get_numpy_points_from_pandas_unscaled(combined_df=combined_df)



########Individual Classifications#######
low_noise_unscaled = get_numpy_points_from_pandas_unscaled(combined_df = low_noise)
topo_water_surface_unscaled = get_numpy_points_from_pandas_unscaled(combined_df = topo_water_surface)
high_noise_unscaled = get_numpy_points_from_pandas_unscaled(combined_df = high_noise)
bathymetry_unscaled = get_numpy_points_from_pandas_unscaled(combined_df = bathymetry)
bathymetric_water_unscaled = get_numpy_points_from_pandas_unscaled(combined_df = bathymetric_water)
derived_water_surface_unscaled = get_numpy_points_from_pandas_unscaled(combined_df = derived_water_surface)
bathy_veg_unscaled = get_numpy_points_from_pandas_unscaled(combined_df =bathy_veg)

The classifications contained in this .las dataset are: 
[0, 7, 9, 18, 40, 41, 42, 80]
Creating a python dictionary containing the entire dataset and individual classifications as separate laspy objects.
Saving classification number: 0
Saving classification number: 7
Saving classification number: 9
Saving classification number: 18
Saving classification number: 40
Saving classification number: 41
Saving classification number: 42
Saving classification number: 80
Available classifications: [0, 7, 9, 18, 40, 41, 42, 80, 1000]
The entire dataset has been selected
All classes have been added to the dataframe.


In [25]:
input_las.xyz

array([[ 2.90988240e+05,  4.81319754e+06, -2.78000000e-01],
       [ 2.90987469e+05,  4.81319772e+06, -2.37000000e-01],
       [ 2.90987360e+05,  4.81319773e+06, -2.67000000e-01],
       ...,
       [ 2.91013483e+05,  4.81304189e+06, -1.35180000e+01],
       [ 2.91018905e+05,  4.81304248e+06, -1.53090000e+01],
       [ 2.91014021e+05,  4.81304163e+06, -1.35940000e+01]])

In [22]:
combined_df

,X,Y,Z,x,y,z,intensity,classification,return_number,number_of_returns,synthetic,key_point,withheld,overlap,scanner_channel,scan_direction_flag,user_data,scan_angle,point_source_id,gps_time
17,290981863,-186802952,-543,290981.863,4813197.048,-0.543,40234,0,1,2,0,0,0,0,0,1,1,-1557,2610,135243.804600
18,290981898,-186803276,-1803,290981.898,4813196.724,-1.803,13772,0,2,2,0,0,0,0,0,1,1,-1557,2610,135243.804600
51,290976250,-186806358,-579,290976.250,4813193.642,-0.579,40878,0,1,2,0,0,0,0,0,1,1,-1481,2610,135243.875758
52,290976273,-186806680,-1841,290976.273,4813193.320,-1.841,13665,0,2,2,0,0,0,0,0,1,1,-1481,2610,135243.875758
145,290993629,-186816313,-1785,290993.629,4813183.687,-1.785,12912,0,2,2,0,0,0,0,0,1,1,-1949,2610,135243.960714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475630,291019886,-186958735,-5806,291019.886,4813041.265,-5.806,4457,80,5,6,0,0,0,0,0,0,4,2643,2812,135839.330419
1475638,291024783,-186958751,-5775,291024.783,4813041.249,-5.775,743,80,4,5,0,0,0,0,0,0,4,2556,2812,135839.330521
1475645,291028322,-186958909,-5577,291028.322,4813041.091,-5.577,1920,80,3,4,0,0,0,0,0,0,4,2492,2812,135839.330593
1475648,291029709,-186958932,-5777,291029.709,4813041.068,-5.777,8561,80,3,4,0,0,0,0,0,0,4,2466,2812,135839.330622


In [5]:
combined_df["X_mean_shift"] = combined_df.X - combined_df.X.mean()
combined_df["Y_mean_shift"] = combined_df.Y - combined_df.Y.mean()
combined_df["Z_mean_shift"] = combined_df.Z - combined_df.Z.mean()

In [12]:
combined_df

,X,Y,Z,intensity,classification,return_number,number_of_returns,synthetic,key_point,withheld,overlap,scanner_channel,scan_direction_flag,user_data,scan_angle,point_source_id,gps_time
2567,290999485,-186802015,-6063,19192,40,1,1,0,0,0,0,0,1,2,-1890,2610,135243.728149
2568,290998268,-186801807,-6001,20705,40,1,1,0,0,0,0,0,1,2,-1864,2610,135243.728178
2570,290997022,-186801595,-5993,19004,40,2,3,0,0,0,0,0,1,2,-1836,2610,135243.728207
2572,290995781,-186801367,-5945,21545,40,2,6,0,0,0,0,0,1,2,-1809,2610,135243.728237
2577,290994546,-186801135,-5915,19130,40,2,2,0,0,0,0,0,1,2,-1781,2610,135243.728266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1475458,291022825,-186959005,-251,65530,42,0,2,0,0,0,0,0,0,4,2610,2812,135839.302463
1475461,291024199,-186959004,-242,65530,42,0,3,0,0,0,0,0,0,4,2585,2812,135839.302492
1475465,291025607,-186959020,-226,65530,42,0,3,0,0,0,0,0,0,4,2560,2812,135839.302521
1475469,291027003,-186959050,-212,65530,42,0,2,0,0,0,0,0,0,4,2534,2812,135839.302550


In [11]:
multiple_classifications_to_las(classified_points=classified_points,class_selections=class_selections)

Created file: 40_41_42
